In [27]:
import numpy as np
import pandas as pd
import datetime as dt
from __future__ import division

In [43]:
# Read in data
filename = 'Algorithms_Data.csv'
data = pd.read_csv(filename)
data = data[-15000:]
train = data.iloc[:-300]
test = data.iloc[-300:]

In [44]:
Y_train = train['Win']
X_train = train[['fran_elo', 'opp_elo', 'RollAvg_A_5_pts', 'RollAvg_B_5_pts', 'RollAvg_A_5_opp_pts', 'RollAvg_B_5_opp_pts', 'Days_Since_Last']]
#X_train = train[['fran_elo', 'opp_elo']]


Y_test = test['Win']
X_test = test[['fran_elo', 'opp_elo', 'RollAvg_A_5_pts', 'RollAvg_B_5_pts', 'RollAvg_A_5_opp_pts', 'RollAvg_B_5_opp_pts', 'Days_Since_Last']]
#X_test = test[['fran_elo', 'opp_elo']]

# Baseline
Guess that the home team loses every game.  If a method doesn't do better than this, it isn't learning much.

In [45]:
print (np.zeros(300) == Y_test).mean()

print

0.573333333333



# SVM

In [ ]:
from sklearn import svm
SVM_model = svm.SVC(C=10**-2, kernel='poly', coef0=1, gamma=10).fit(X_train,Y_train)
preds = SVM_model.predict(X_test)
prob = np.sum(preds == Y_test)/len(Y_test)
print prob
print preds
# Holy slow

# Logistic Regression

In [33]:
from sklearn import linear_model as lm

lam = 10**2

sk_logreg = lm.LogisticRegression(C=np.abs(1/lam))
sk_logreg.fit(X_train, Y_train)
preds = sk_logreg.predict(X_test)
print (preds == Y_test).mean()
# Surpsingly(maybe not) good, very fast
print preds

0.716666666667
[ True  True  True  True False False False  True False  True  True False
 False False False False False  True False False  True False  True  True
  True  True  True  True  True  True  True False False  True  True  True
 False False  True False False False  True False False False  True  True
  True False False  True False False  True False  True  True  True False
 False  True  True False False  True  True  True False  True False  True
 False False False False False False  True False False False  True  True
 False False False  True  True False  True  True False  True  True False
 False False  True  True False False False False False  True False False
 False  True False  True False False False False False False  True False
  True  True  True False False  True False False  True  True  True  True
 False  True  True False False False False  True  True False  True  True
 False  True False  True  True  True  True False  True False False False
 False False  True False False False

# Ridge Regression 

In [34]:
from sklearn.linear_model import Ridge as ri

skRidge = ri()
skRidge.fit(X_train, Y_train)
preds = skRidge.predict(X_test) > .5
print (preds == Y_test).mean()
print preds

0.726666666667
[ True  True  True  True False False False  True False  True  True False
 False False False False False  True False False  True False  True  True
  True  True  True  True  True  True  True False False  True  True  True
 False False  True False False False  True False False False  True  True
  True False False  True False False  True False  True False  True False
 False  True  True False False  True  True  True False  True False  True
 False False False False False False  True False False False  True  True
 False False False  True  True False  True  True False  True  True False
 False False  True False False False False False False  True False False
 False  True False  True False False False False False False  True False
  True  True  True False False  True False False  True  True  True  True
 False  True  True False False False False  True False False  True  True
 False  True False  True  True  True  True False  True False False False
 False False  True False False False

# Gradient Boost 

In [35]:
from sklearn.ensemble import GradientBoostingClassifier as GBC

gbc = GBC(max_leaf_nodes=200, min_weight_fraction_leaf=0.5,min_samples_split=54).fit(X_train,Y_train)
preds = gbc.predict(X_test)
print (preds == Y_test).mean()
print preds

# Fine tuning seems to have minimal effect

0.583333333333
[ True False False False False False False False False False False False
 False False False False False  True False False False False False  True
  True  True  True  True  True  True False False  True  True  True False
 False False False False  True False  True False False False False False
  True False False  True False False  True False  True  True False False
 False False  True False False  True False  True False False False False
 False False False False False False  True False False False  True  True
 False False False  True False False False False False  True False  True
 False False  True False False False False False False False  True False
  True  True False  True False False False False False False  True False
  True  True False False False False False False False False False False
 False False False False False False False False  True False  True False
  True  True  True  True  True False  True False False False False False
  True  True False False False False

# MLP

In [36]:
from sklearn.neural_network import MLPClassifier as mlp

gbm = mlp(solver="lbfgs", activation='tanh', early_stopping=True).fit(X_train,Y_train)
print (gbm.predict(X_test) == Y_test).mean()

# Super slow, fine tuning doesnt seem to help much

0.7


# Random Forests

In [37]:
from sklearn.ensemble import RandomForestClassifier as RFC

rfc = RFC(min_samples_split=3, n_estimators =100).fit(X_train,Y_train)
print (rfc.predict(X_test) == Y_test).mean()

# Fine tuned parameters, this is the best I could get

0.676666666667


# Decision Tree

In [38]:
from sklearn.tree import DecisionTreeClassifier as DTC

dtc = DTC(criterion='entropy').fit(X_train,Y_train)
preds = dtc.predict(X_test)
print (preds == Y_test).mean()
print preds

0.636666666667
[ True False  True False False False  True  True  True  True  True False
  True  True  True False  True  True False  True  True False  True  True
  True False  True False False False False  True False  True  True  True
 False  True False  True  True False  True False False False  True  True
  True False False False  True  True  True False False  True  True False
 False  True  True False False False False  True False False False  True
 False False False False False False  True False False False False  True
 False False False  True False False False  True False  True  True False
 False False  True  True False False False False False False False False
 False  True  True  True False False False  True False  True False False
  True False False False False  True False  True  True False False False
 False False False False  True False False  True  True False  True  True
 False False False  True  True False  True False False False  True False
 False False  True False  True False

# XGBoost

In [88]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, Y_train)
dtest = xgb.DMatrix(X_test)

bst = xgb.train({}, dtrain)
preds = bst.predict(dtest) > .5
print (preds == Y_test).mean()
print preds


0.62
[ True  True False False False False False False False False False False
 False False False False False  True False False False False  True  True
  True  True  True  True  True False False False  True  True False  True
 False False  True  True  True False False False False False False  True
  True False False  True False False  True False  True  True  True False
 False  True False False False False  True  True False  True False False
 False False False False False False  True  True False False  True  True
 False False False  True  True False False False False False False  True
 False False  True  True False False False False  True False  True False
 False  True False  True False False False False False False False False
  True  True False False False  True False False  True False False  True
 False False  True False False False False False  True  True  True False
  True  True  True  True  True False  True False False  True False False
  True  True  True False False False False  Tr

# Algorithms we chose not to use

### Nearest neighbor

Due to the fact that we are not performing any kind of cluster analysis, we have decided the NN-classifying and NN-regression algorithms are not useful. 

### Gaussian discriminant analysis

Similar to the issues with nearest neighbor, since we are not doing any kind of classification, GDA will not be useful with our data set or our project.

### Mixture models with latent variables (train with EM)

Because there is not an unknown distribution in our dataset, mixture models along with EM will not work with our data set.


### Kalman Filters

Our data is not a true time series because we are not interested in how teams' overall performances changes over time. Games are decided strictly by winners and losers, so there is not any measurable error or noise. Because of this we have determined that it will not be useful in our methods. 

### AR, MA, ARMA, ARIMA time series models

Because these models are trying to describe certain time-varying processes of a time series, they will not be helpful in trying to predict wins and losses of specific games. 